In [ ]:
!pip install fuzzywuzzy

In [ ]:
!pip install python-Levenshtein

In [ ]:
!pip install gensim

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download fr_core_news_lg

In [ ]:
!pip install --upgrade pandas

In [ ]:
!pip install modin

In [ ]:
!pip install ray

In [11]:
import ray
ray.init()

2023-08-29 08:46:03,636	INFO worker.py:1621 -- Started a local Ray instance.


Python version:,3.10.12
Ray version:,2.6.3


# Importing :


1.   **pandas** and **numpy** are used for data manipulation and analysis.
2. **json** is imported for working with JSON data.
3. **gc** stands for the garbage collector, which helps manage memory.
4. **os** provides functions for interacting with the operating system.
5. **time** is used for time-related operations.
6. **re** allows working with regular expressions.
7. **nltk** is the Natural Language Toolkit for text processing.
8. **logging** is for generating log messages.
9. **spacy** is a library for natural language processing.



In [12]:
import numpy as np
import json
import gc
import os
import time
import re
import nltk
import logging
import spacy
os.environ['__MODIN_AUTOIMPORT_PANDAS__'] = '1'
import modin.pandas as pd

1. **fuzzywuzzy** provides **fuzzy** string matching capabilities.
2. **TfidfVectorizer** and **CountVectorizer** from **sklearn**.feature_extraction.text are used for text vectorization.
3. **cosine_similarity** from **sklearn.metrics.pairwise** calculates cosine similarity between vectors.
4. **ProcessPoolExecutor** from **concurrent.futures** enables parallel processing.
5. **tqdm** is a library for creating progress bars during iteration.
6. **Word2Vec**, **WmdSimilarity**, and **KeyedVectors** from **gensim.models** are for word embedding and similarity calculations.
7. **google.colab** contains utilities for working in Google Colab environment.
8. **stopwords** and **WordNetLemmatizer** from **nltk.corpus** are used for text preprocessing.
9. **List** and Union from typing module provide type hints for function parameters.
These imports allow you to leverage various functionalities for text processing, analysis, and parallel processing.

In [13]:
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from gensim.models import Word2Vec
from gensim.similarities import WmdSimilarity
from gensim.models import KeyedVectors
from google.colab import files
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from typing import List, Union

# Loading:

In [14]:
nlp = spacy.load('fr_core_news_lg')
# Initialize logging
logging.basicConfig(level=logging.INFO)

# Define the folder path
folder_path = '/content/excelwork/'

# List all files in the folder
all_files = os.listdir(folder_path)

# Filter out Excel files
excel_files = [f for f in all_files if f.endswith('.xlsx') or f.endswith('.xls')]

# Full paths to Excel files
excel_file_paths = [os.path.join(folder_path, f) for f in excel_files]

In [15]:
standard_columns = np.load('/content/standard_columns.npy')
Type_TP = np.load('/content/Type_TP.npy')
INTERVENANT_OPTIONS = np.load('/content/INTERVENANT_OPTIONS.npy')
Typologie = np.load('/content/Typologie.npy')
Causes_Echecs = np.load('/content/Causes_Echecs.npy')
ADMIN_NAMES = np.load('/content/ADMIN_NAMES.npy')
OPTIONS_MAP_Stringheader = np.load('/content/OPTIONS_MAP_Stringheader.npy', allow_pickle=True)

In [16]:
output_xlsx_file = "/content/combined_data.xlsx"

In [17]:
# Initialize logging with timestamp and file handler
logging.basicConfig(filename='excel_processing.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Combining Excel file Part:

## Table of Contents
1. [worker](#worker)
2. [parallel_concat](#parallel_concat)
3. [find_similar_column](#find_similar_column)
4. [check_and_fill_missing_columns](#check_and_fill_missing_columns)
5. [process_combined_df](#process_combined_df)
6. [find_similar_file_path](#find_similar_file_path)
7. [process_excel_file](#process_excel_file)
8. [combine_excel_files_to_xlsx](#combine_excel_files_to_xlsx)


### `worker`

**Description**:  
Concatenates a list of DataFrames and removes rows that are all spaces or NaNs.

### `parallel_concat`

**Description**:  
Uses parallel processing to concatenate chunks of DataFrames.

### `find_similar_column`

**Description**:  
Finds the most similar column name to a given reference column.

### `check_and_fill_missing_columns`

**Description**:  
Checks for missing columns in a DataFrame and optionally fills them.

### `process_combined_df`

**Description**:  
Combines a list of DataFrames into a single DataFrame and saves it to Excel.

### `find_similar_file_path`

**Description**:  
Finds the most similar file path to a given target path.

### `process_excel_file`

**Description**:  
Processes an Excel file into a DataFrame.

### `combine_excel_files_to_xlsx`

**Description**:  
Combines multiple Excel files into a single Excel file.

## `worker`

### Description
The `worker` function concatenates a list of pandas DataFrames and performs some cleaning operations.

### Parameters
- **df_list (list of pd.DataFrame)**:  
  List of DataFrames to be concatenated.

### Returns
- **pd.DataFrame**:  
  A single DataFrame obtained by concatenating all the DataFrames in `df_list`. The DataFrame also undergoes the following cleaning operations:
  - Rows containing only spaces or NaNs are removed.
  - The index is reset.

### Example
```python
df1 = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
df2 = pd.DataFrame({'A': [5, 6], 'B': [7, 8]})
result = worker([df1, df2])


In [18]:
def worker(df_list: List[pd.DataFrame]) -> Union[pd.DataFrame, None]:
    """
    Concatenates a list of DataFrames, removes rows that are all spaces or NaNs, and resets the index.
    Optimized for memory usage.

    Parameters:
    - df_list (List[pd.DataFrame]): List of DataFrames to concatenate.

    Returns:
    - pd.DataFrame or None: The concatenated DataFrame or None if an error occurs.
    """
    try:
        # Initialize logging
        logging.basicConfig(level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

        # Step 1: Concatenate DataFrames and reset the index
        logging.info("Concatenating DataFrames...")
        concatenated_df = pd.concat(df_list, ignore_index=True)

        # Clear the list to free memory
        del df_list
        gc.collect()

        # Step 2: Remove rows that are all spaces or NaNs
        logging.info("Removing rows with all spaces or NaNs...")
        concatenated_df = concatenated_df[~concatenated_df.apply(lambda row: all(str(cell).isspace() or pd.isna(cell) for cell in row), axis=1)]

        # Step 3: Reset the index again after removing rows
        logging.info("Resetting index...")
        concatenated_df.reset_index(drop=True, inplace=True)

        # Step 4: Convert columns to more memory-efficient data types
        logging.info("Optimizing data types...")
        for col in concatenated_df.columns:
            concatenated_df[col] = pd.to_numeric(concatenated_df[col], errors='ignore', downcast='integer')

        logging.info("DataFrame successfully processed.")
        return concatenated_df

    except Exception as e:
        logging.error(f"An error occurred while processing the DataFrame: {e}")
        return None

## `parallel_concat`

### Description
The `parallel_concat` function performs parallel concatenation of a list of pandas DataFrames. It divides the list into smaller chunks and uses multiprocessing to concatenate them.

### Parameters
- **dataframes (list of pd.DataFrame)**:  
  List of DataFrames to be concatenated.

### Returns
- **pd.DataFrame**:  
  A single DataFrame obtained by concatenating all the DataFrames in the input list. The DataFrame has its index reset.

### How It Works
1. Divides the list of DataFrames into chunks of 5.
2. Uses `ProcessPoolExecutor` for parallel processing of each chunk.
3. Each chunk is processed by the `worker` function to concatenate the DataFrames in that chunk.
4. Finally, concatenates all the chunks to form a single DataFrame.

### Example
```python
df1 = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
df2 = pd.DataFrame({'A': [5, 6], 'B': [7, 8]})
df3 = pd.DataFrame({'A': [9, 10], 'B': [11, 12]})
result = parallel_concat([df1, df2, df3])

In [19]:
def parallel_concat(dataframes: List[pd.DataFrame]) -> pd.DataFrame:
    """
    Concatenates a list of DataFrames in parallel, optimizing for memory usage.

    Parameters:
    - dataframes (List[pd.DataFrame]): List of DataFrames to concatenate.

    Returns:
    - pd.DataFrame: The concatenated DataFrame.
    """
    # Initialize logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

    # Step 1: Divide the list of DataFrames into chunks of 5
    logging.info("Dividing DataFrames into chunks...")
    chunks = [dataframes[i:i + 5] for i in range(0, len(dataframes), 5)]

    concatenated_dfs = []

    # Step 2: Use ProcessPoolExecutor for parallel processing
    logging.info("Starting parallel processing...")
    with ProcessPoolExecutor() as executor:
        for concatenated_chunk in executor.map(worker, chunks):
            concatenated_dfs.append(concatenated_chunk)

            # Clear the chunk to free memory
            del concatenated_chunk
            gc.collect()

    # Step 3: Concatenate all the chunks and reset the index
    logging.info("Concatenating all chunks...")
    final_df = pd.concat(concatenated_dfs, ignore_index=True)

    # Clear the list to free memory
    del concatenated_dfs
    gc.collect()

    # Step 4: Convert columns to more memory-efficient data types
    logging.info("Optimizing data types...")
    for col in final_df.columns:
        final_df[col] = pd.to_numeric(final_df[col], errors='ignore', downcast='integer')

    logging.info("DataFrames successfully concatenated.")
    return final_df

In [20]:
# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

## `find_similar_column`

### Description
The `find_similar_column` function finds the most similar column name to a given reference column name using fuzzy string matching. It uses the FuzzyWuzzy library to calculate the similarity ratio.

### Parameters
- **reference_col (str)**:  
  The reference column name to which other column names will be compared.
  
- **columns (list of str)**:  
  List of column names to compare against the reference column.
  
- **similarity_threshold (int, optional)**:  
  The minimum similarity ratio required to consider a column as similar. Default is 80.

### Returns
- **str or None**:  
  Returns the most similar column name if the similarity ratio is above the threshold, otherwise returns `None`.

### How It Works
1. Initializes `max_similarity` to 0 and `most_similar_col` to `None`.
2. Iterates through each column in the `columns` list.
3. Calculates the similarity ratio between `reference_col` and each column.
4. Updates `most_similar_col` if a higher similarity ratio is found.
5. Returns `most_similar_col` if its similarity ratio is above the `similarity_threshold`.

### Example
```python
reference_col = "Name"
columns = ["name", "full_name", "first_name"]
result = find_similar_column(reference_col, columns)


In [21]:
def find_similar_column(reference_col: str, columns: list, similarity_threshold: int = 80) -> str:
    """
    Finds the most similar column name to a given reference column name.

    Parameters:
    - reference_col (str): The reference column name.
    - columns (list): List of existing column names.
    - similarity_threshold (int): The minimum similarity ratio to consider a match.

    Returns:
    - str: The most similar column name if found, otherwise None.
    """
    # Initialize logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

    max_similarity = 0
    most_similar_col = None

    # Precompute the lowercase version of reference_col
    reference_col_lower = reference_col.lower()

    for i, col in enumerate(columns):
        col_lower = str(col).lower()

        # Calculate similarity
        similarity = fuzz.ratio(reference_col_lower, col_lower)

        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_col = col

            # Log the new maximum similarity found
            logging.info(f"New max similarity {max_similarity} found at index {i} with column {col}.")

            # Early exit if 100% similarity
            if max_similarity == 100:
                logging.info("100% similarity found, exiting loop.")
                break

    if max_similarity > similarity_threshold:
        logging.info(f"Most similar column found: {most_similar_col} with similarity {max_similarity}.")
        return most_similar_col
    else:
        logging.warning(f"No similar column found with similarity above {similarity_threshold}.")
        return None

## `check_and_fill_missing_columns` Function Documentation

### Description
The `check_and_fill_missing_columns` function checks for missing columns in a DataFrame and optionally fills them with `pd.NA` values. It also looks for similar column names using fuzzy string matching if a column is missing.

### Parameters
- **df (DataFrame)**:  
  The DataFrame to check for missing columns.
  
- **standard_columns (list of str)**:  
  List of standard column names that should be present in the DataFrame.
  
- **fill_missing (bool)**:  
  Whether to fill missing columns with `pd.NA` values. Default is `True`.
  
- **similarity_threshold (int)**:  
  The minimum similarity ratio required to consider a column as similar. Default is 80.

### Returns
- **list of str**:  
  Returns a list of missing columns that were not found in the DataFrame and for which no similar columns were found.

### How It Works
1. Initializes an empty list called `missing_columns`.
2. Iterates through each column in the `standard_columns` list.
3. Checks if the column is present in the DataFrame.
4. If not, it uses `find_similar_column` to look for a similar column.
5. If a similar column is found and `fill_missing` is `True`, it fills the DataFrame with `pd.NA` values for that column.
6. If no similar column is found, it adds the column to `missing_columns`.
7. Returns `missing_columns`.

### Example
```python
df = pd.DataFrame({'name': [1, 2], 'age': [3, 4]})
standard_columns = ['name', 'age', 'gender']
result = check_and_fill_missing_columns(df, standard_columns, True, 80)

In [22]:

def check_and_fill_missing_columns(df: pd.DataFrame, standard_columns: list, fill_missing: bool = True, similarity_threshold: int = 80) -> list:
    """
    Checks for missing columns in the DataFrame and fills them if specified.

    Parameters:
    - df (pd.DataFrame): The DataFrame to check.
    - standard_columns (list): List of standard column names.
    - fill_missing (bool): Whether to fill missing columns with NA values.
    - similarity_threshold (int): The minimum similarity ratio to consider a match.

    Returns:
    - list: List of missing columns.
    """
    # Initialize logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

    # Convert to sets for faster lookup
    existing_columns = set(df.columns)
    standard_columns_set = set(map(str, standard_columns))

    # Initialize list to store missing columns
    missing_columns = []

    for col in standard_columns_set:
        if col not in existing_columns:
            similar_col = find_similar_column(col, list(existing_columns), similarity_threshold)
            logging.info(f"Similar column found: {similar_col}.")

            if similar_col:
                logging.warning(f"Missing column in the data, but a similar column found: {col} (similar to {similar_col}).")

                if fill_missing:
                    df[col] = pd.NA
                    logging.debug(f"Missing column {col} filled with NA.")
            else:
                logging.warning(f"Missing column in the data and no similar column found: {col}.")
                missing_columns.append(col)

    return missing_columns

## `process_combined_df`

### Description
The `process_combined_df` function combines a list of DataFrames into a single DataFrame using parallel processing. It then saves the combined DataFrame to an Excel file.

### Parameters
- **dataframes (list of DataFrames)**:  
  The list of DataFrames to be combined.
  
- **output_xlsx_file (str)**:  
  The name of the output Excel file where the combined DataFrame will be saved.

### Returns
- **DataFrame**:  
  Returns the combined DataFrame if successful, otherwise returns `None`.

### How It Works
1. Checks if the `dataframes` list is empty. If it is, logs an error and returns `None`.
2. Calls the `parallel_concat` function to concatenate the DataFrames in parallel.
3. Saves the combined DataFrame to an Excel file specified by `output_xlsx_file`.
4. Logs a success message.

### Example
```python
df1 = pd.DataFrame({'name': ['Alice', 'Bob'], 'age': [30, 40]})
df2 = pd.DataFrame({'name': ['Charlie', 'David'], 'age': [50, 60]})
dataframes = [df1, df2]
output_xlsx_file = "combined_data.xlsx"

result = process_combined_df(dataframes, output_xlsx_file)

In [23]:
def process_combined_df(dataframes: list, output_xlsx_file: str) -> pd.DataFrame:
    """
    Processes a list of DataFrames, combines them, and saves the result to an Excel file.

    Parameters:
    - dataframes (list): List of DataFrames to combine.
    - output_xlsx_file (str): Path to the output Excel file.

    Returns:
    - pd.DataFrame: The combined DataFrame.
    """
    # Initialize logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

    if not dataframes:
        logging.error("No DataFrames to combine.")
        return None

    # Using parallel processing for concatenation
    logging.info("Starting to combine DataFrames...")
    final_df = parallel_concat(dataframes)

    # Save to Excel in a memory-efficient way
    print(output_xlsx_file)
    with pd.ExcelWriter(output_xlsx_file, engine='openpyxl', mode='a') as writer:
        final_df.to_excel(writer, index=False)

    logging.info("Successfully combined DataFrames and saved to Excel.")

    return final_df

## `find_similar_file_path`
### Description
The `find_similar_file_path` function searches for a file path that is similar to the given `target_path` among a list of `existing_paths`. It uses NLP-based similarity scoring to find the most similar path.

### Parameters
- **target_path (str)**:  
  The file path you're trying to find a match for.
  
- **existing_paths (list of str)**:  
  The list of existing file paths to search through.

- **similarity_threshold (float, optional)**:  
  The minimum similarity score to consider a path as similar. Default is 0.8.

### Returns
- **str or None**:  
  Returns the most similar file path if a match is found, otherwise returns `None`.

### How It Works
1. Converts the `target_path` and each path in `existing_paths` to NLP objects.
2. Calculates the similarity score between `target_path` and each existing path.
3. If a path with a similarity score above the `similarity_threshold` is found, it is returned.
4. If no such path is found, returns `None`.

### Example
```python
target_path = "/home/user/documents/file1.txt"
existing_paths = ["/home/user/documents/file2.txt", "/home/user/downloads/file1.txt"]
similarity_threshold = 0.8

result = find_similar_file_path(target_path, existing_paths, similarity_threshold)

In [24]:
def find_similar_file_path(target_path: str, existing_paths: list, similarity_threshold: float = 0.75) -> str:
    """
    Finds the most similar file path to the target path from a list of existing paths.

    Parameters:
    - target_path (str): The target file path.
    - existing_paths (list): List of existing file paths.
    - similarity_threshold (float): The similarity threshold for considering paths as similar.

    Returns:
    - str: The most similar file path, or None if no path is similar enough.
    """
    # Initialize logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

    target_doc = nlp(target_path)
    max_similarity = 0
    most_similar_path = None

    # Batch processing for efficiency
    path_docs = list(nlp.pipe(existing_paths))

    for path, path_doc in zip(existing_paths, path_docs):
        similarity = target_doc.similarity(path_doc)

        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_path = path

            # Early exit if similarity is very high
            if max_similarity > 0.95:
                logging.info(f"High similarity found. Exiting early.")
                break

    if max_similarity > similarity_threshold:
        return most_similar_path
    else:
        logging.warning(f"No similar file path found with a similarity above {similarity_threshold}.")
        return None

## `process_excel_file`

### Description
The `process_excel_file` function reads an Excel file, validates its columns against a list of `standard_columns`, and fills missing columns if needed. It also handles file path issues by searching for similar existing file paths.

### Parameters
- **file (str)**:  
  The path to the Excel file to be processed.
  
- **standard_columns (list of str)**:  
  The list of standard column names that the DataFrame should have.
  
- **fill_missing (bool)**:  
  Whether to fill missing columns with `pd.NA`. Default is `True`.

- **similarity_threshold (int)**:  
  The minimum similarity score to consider a column as similar. Default is 80.

### Returns
- **tuple (DataFrame, bool)**:  
  Returns a tuple containing the processed DataFrame and a success status (`True` or `False`).

### How It Works
1. Validates the file path and searches for a similar existing file path if the file does not exist.
2. Reads the Excel file into a DataFrame.
3. Checks for missing columns and fills them if `fill_missing` is `True`.
4. Reindexes the DataFrame based on `standard_columns`.
5. Returns the processed DataFrame and a success status.

### Example
```python
file = "data.xlsx"
standard_columns = ["Name", "Age", "Gender"]
fill_missing = True
similarity_threshold = 80

result = process_excel_file(file, standard_columns, fill_missing, similarity_threshold)

In [25]:
def process_excel_file(file, standard_columns, fill_missing, similarity_threshold):
    try:
        # Validate file path
        if not os.path.exists(file):
            logging.warning(f"File does not exist: {file}")

            # Search for a similar existing file path
            existing_files = [f for f in os.listdir('.') if os.path.isfile(f)]
            logging.info('Found files: %s', existing_files)
            similar_file = find_similar_file_path(file, existing_files)
            logging.info("Similar file found: " + similar_file)

            if similar_file:
                logging.info(f"Did you mean: {similar_file}? Proceeding with this file.")
                file = similar_file
            else:
                logging.error("No similar file found.")
                return (None, False)  # Ensure a tuple is returned

        # Read and process the Excel file
        df = pd.read_excel(file, engine='openpyxl')  # Specify the engine
        logging.info('Read {} rows from Excel file'.format(len(df)))
        df = df.dropna(how='all')  # Remove rows that are all NaNs
        logging.debug('Done dropping rows')

        # Handle date columns
        date_columns = [col for col in df.columns if pd.api.types.is_datetime64_any_dtype(df[col])]
        print("Date columns: ", date_columns)

        for col in date_columns:
            df[col] = df[col].dt.strftime('%Y-%m-%d')  # Convert date to string format
            logging.debug(f"Date format changed for {col}.")

        missing_columns = check_and_fill_missing_columns(df, standard_columns, fill_missing, similarity_threshold)
        if missing_columns:
            logging.warning(f"Missing columns in {file}: {missing_columns}.")
            if fill_missing:
                for col in missing_columns:
                    df[col] = pd.NA
                    logging.debug(f"Missing column {col} filled with NA.")
        df = df.reindex(columns=standard_columns)
        logging.debug(f"Reindexed columns in {file}.")
        return (df, True)  # Return the DataFrame and success status

    except Exception as e:
        logging.error(f"Error processing Excel file {file}: {e}")
        return (None, False)  # Ensure a tuple is returned

## `combine_excel_files_to_xlsx`

### Description
The `combine_excel_files_to_xlsx` function combines multiple Excel files into a single Excel file. It validates each file, processes it, and then combines them. It also provides a summary report of the operation.

### Parameters
- **excel_files (List[str])**:  
  The list of paths to the Excel files to be combined.
  
- **standard_columns (List[str])**:  
  The list of standard column names that each DataFrame should have.
  
- **output_xlsx_file (str)**:  
  The path where the combined Excel file will be saved.
  
- **fill_missing (bool)**:  
  Whether to fill missing columns with `pd.NA`. Default is `True`.

- **similarity_threshold (int)**:  
  The minimum similarity score to consider a column as similar. Default is 80.

### Returns
- **tuple (DataFrame, dict)**:  
  Returns a tuple containing the combined DataFrame and a summary report.

### How It Works
1. Validates the input parameters.
2. Initializes a summary report.
3. Uses `ProcessPoolExecutor` for parallel processing of Excel files.
4. Updates the summary report based on the success or failure of processing each file.
5. Combines all the processed DataFrames.
6. Saves the combined DataFrame to an Excel file.
7. Returns the combined DataFrame and the summary report.

### Example
```python
excel_files = ["file1.xlsx", "file2.xlsx"]
standard_columns = ["Name", "Age", "Gender"]
output_xlsx_file = "combined.xlsx"
fill_missing = True
similarity_threshold = 80

result = combine_excel_files_to_xlsx(excel_files, standard_columns, output_xlsx_file, fill_missing, similarity_threshold)

In [26]:
def combine_excel_files(excel_files, standard_columns, fill_missing, similarity_threshold, output_file_path, output_xlsx_file):
    #START_CODE
    try:
        # Initialize logging
        logging.basicConfig(level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

        logging.info("Starting to process Excel files...")

        if len(excel_files) == 0 or standard_columns.size == 0 or not output_xlsx_file:
            logging.error("Invalid input parameters.")
            return None

        summary_report = {"processed": 0, "skipped": 0, "missing_columns": []}
        dataframes = []

        with ProcessPoolExecutor() as executor:
            with tqdm(total=len(excel_files), desc="Processing files") as pbar:
                for df, success in executor.map(process_excel_file, excel_files, [standard_columns]*len(excel_files), [fill_missing]*len(excel_files), [similarity_threshold]*len(excel_files)):
                    pbar.update(1)
                    if success:
                        dataframes.append(df)
                        summary_report["processed"] += 1
                        logging.info(f"Successfully processed file. Total processed so far: {summary_report['processed']}")
                    else:
                        summary_report["skipped"] += 1
                        logging.info(f"Skipped a file. Total skipped so far: {summary_report['skipped']}")

        if len(dataframes) == 0:
            logging.error("No Excel files were processed. Exiting...")
            return None, None

        logging.info("Successfully processed Excel files. Starting to combine them...")

        combined_df = process_combined_df(dataframes, output_xlsx_file)

        if not combined_df:
            logging.error("Error combining Excel files. Exiting...")
            return None, None

        logging.info("Successfully combined Excel files and saved to XLSX!")
        logging.info(f"Summary Report: {summary_report}")

        combined_df.to_excel(output_file_path, index=False)

        return combined_df, summary_report

    except Exception as e:
        logging.error(f"Error combining Excel files: {e}")
        return None, None

In [27]:
import warnings
from openpyxl import Workbook
warnings.simplefilter("ignore", category=DeprecationWarning)

In [ ]:
# Call the combine_excel_files_to_xlsx function
result = combine_excel_files(excel_file_paths, standard_columns, fill_missing=True,output_file_path='/content/', output_xlsx_file = 'combine_data.xlsx', similarity_threshold = 0.75)

# Check if the result is not None
if result is not None:
    combined_df, summary_report = result

    # Print the result tuple for debugging
    print("Result:", result)

    # Check if there are any errors in the summary_report
    if summary_report and summary_report["skipped"] == 0:
        # Now, you can safely use head() on the combined DataFrame
        print(combined_df.head())
    else:
        print("There were errors or skipped files in combining Excel files.")
else:
    # Handle the case where there was an error in combining the files
    print("There was an error in combining Excel files.")

Processing files:   0%|          | 0/13 [00:00<?, ?it/s]UserWarning: When using a pre-initialized Ray cluster, please ensure that the runtime env sets environment variable __MODIN_AUTOIMPORT_PANDAS__ to 1


In [ ]:
df = combined_df.copy()

In [ ]:
manual_replacements = {
    "FTTH":'ZMD SFR',
    "FTTB":'ZMD SFR',
    "B2B ACCES": 'B2B',
    "SQUALUM":'ZMD SFR',
    "TRANSFO CABLE":'ZMD SFR'
}

In [ ]:
 #Configure logging
logging.basicConfig(filename='matching_log.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
nlp = spacy.load('fr_core_news_lg')

# Transforming Data Part :

# Text Similarity Analysis Toolbox

This toolbox provides a set of functions for analyzing text similarity, comparing values, and processing text data for natural language analysis.

## `remove_numbers_from_value(input_value)`

Remove numbers from a given input value.

### Parameters:

- `input_value` (numeric or string): The input value from which numbers will be removed.

### Output:

The function returns a string where all numeric characters have been removed from the `input_value`.

## `preprocess(text)`

Preprocess text data for natural language analysis.

### Parameters:

- `text` (string): The input text to be preprocessed.

### Output:

The function returns a processed string that has undergone preprocessing steps, including removing telephone numbers, tokenization, and stopword removal.

## `jaccard_similarity(str1, str2)`

Calculate the Jaccard similarity between two strings.

### Parameters:

- `str1` (string): The first input string.
- `str2` (string): The second input string.

### Output:

The function returns a value between 0 and 1, representing the Jaccard similarity between the input strings.

## `compute_jaccard_similarities(processed_value, compare_array)`

Compute Jaccard similarities between a processed value and an array of comparison texts.

### Parameters:

- `processed_value` (string): The processed value for which Jaccard similarities will be computed against the comparison texts.
- `compare_array` (array-like): An array of comparison texts.

### Output:

The function returns a NumPy array containing Jaccard similarity scores between the `processed_value` and each text in the `compare_array`.


## `summary_report(results_df, threshold)`

Generate a summary report based on the results from the provided DataFrame.

### Parameters:

- `results_df` (DataFrame): The DataFrame containing the results to be summarized.
- `threshold` (numeric): The threshold used for matching values.

### Description:

This function generates a summary report based on the provided DataFrame `results_df` and the specified `threshold`. It calculates and displays various statistics related to the matched and unmatched values in the DataFrame.

### Parameters:

- `results_df` (DataFrame): The DataFrame containing the results to be summarized.
- `threshold` (numeric): The threshold used for matching values.

### Output:

The function prints out the following information to the console:

- Total rows processed: The total number of rows in the `results_df`.
- Threshold used: The specified threshold value.
- Total matched values: The count of matched values in the `results_df`.
- Total unmatched values: The count of unmatched values in the `results_df`.

If the `results_df` is `None`, a warning message will be logged indicating that there are no results to report. If an error occurs during the process, an error message will be logged with details about the error.

### Example Usage:

```python
import pandas as pd

# Create a sample DataFrame
data = {'matched_value': [10, 20, None, 30, 40]}
results_df = pd.DataFrame(data)

# Call the summary_report function
threshold = 25
summary_report(results_df, threshold)


In [ ]:
def summary_report(results_df, threshold):
    try:
        if results_df is not None:
            total_rows = results_df.shape[0]
            matched_values = results_df['matched_value'].count()
            unmatched_values = results_df['matched_value'].isna().sum()

            logging.info("Summary Report:")
            logging.info(f"Total rows processed: {total_rows}")
            logging.info(f"Threshold used: {threshold}")
            logging.info(f"Total matched values: {matched_values}")
            logging.info(f"Total unmatched values: {unmatched_values}")
        else:
            logging.warning("Summary Report: No results to report.")
    except Exception as e:
        logging.error(f"An error occurred while generating the summary report: {str(e)}")

## `remove_numbers_from_value(input_value)`

Remove numbers from a given input value.

### Parameters:

- `input_value` (numeric or string): The input value from which numbers will be removed.

### Description:

This function takes an input value, which can be either a numeric value or a string containing numeric characters. It then removes all numeric characters from the input value and returns the result as a string.

### Parameters:

- `input_value` (numeric or string): The input value from which numbers will be removed.

### Output:

The function returns a string where all numeric characters have been removed from the `input_value`.

### Example Usage:

```python
# Call the remove_numbers_from_value function
input_value = "Hello123World456"
result = remove_numbers_from_value(input_value)
print(result)  # Output: "HelloWorld"


In [ ]:
def remove_numbers_from_value(input_value):
    # Convert the input_value to a string
    input_str = str(input_value)

    # Initialize an empty string to store the result
    result_str = ""

    # Loop through each character in the input string
    for char in input_str:
        # Check if the character is not a digit (i.e., not a number)
        if not char.isdigit():
            # If it's not a number, add it to the result string
            result_str += char

    return result_str

## `preprocess(text)`

Preprocess text data for natural language analysis.

### Parameters:

- `text` (string): The input text to be preprocessed.

### Description:

This function takes a text input and performs preprocessing steps to prepare it for natural language analysis. It removes telephone numbers, tokenizes the text using spaCy for proper natural language handling, and removes stopwords.

### Parameters:

- `text` (string): The input text to be preprocessed.

### Output:

The function returns a processed string that has undergone the following preprocessing steps:

1. Removing telephone numbers using regular expressions.
2. Tokenizing the text using spaCy to handle natural language effectively.
3. Removing stopwords from the tokenized text.

### Example Usage:

```python
import spacy

# Load the spaCy language model
nlp = spacy.load("en_core_web_sm")

# Call the preprocess function
input_text = "Hello, my phone number is 123-456-7890."
processed_result = preprocess(input_text)
print(processed_result)  # Output: "Hello , phone number ."


In [ ]:

def preprocess(text):
    if pd.isnull(text) or text is np.nan:  # Checking for NaN values
        return ''

    # Use regular expressions to remove telephone numbers
    text_str = remove_numbers_from_value(text)
    print(text_str)
    # Tokenize using spaCy to handle natural language properly
    tokens = nlp(text_str)
    processed_text = ' '.join([token.text for token in tokens if not token.is_stop])  # Remove stopwords
    return processed_text


## `jaccard_similarity(str1, str2)`

Calculate the Jaccard similarity between two strings.

### Parameters:

- `str1` (string): The first input string.
- `str2` (string): The second input string.

### Description:

This function calculates the Jaccard similarity between two input strings `str1` and `str2`. Jaccard similarity is a measure of how similar two sets are, and it is calculated as the size of the intersection of the sets divided by the size of their union.

### Parameters:

- `str1` (string): The first input string.
- `str2` (string): The second input string.

### Output:

The function returns a value between 0 and 1, representing the Jaccard similarity between the input strings. A higher value indicates greater similarity.

### Example Usage:

```python
# Call the jaccard_similarity function
string1 = "apple banana orange"
string2 = "banana orange kiwi"
similarity_score = jaccard_similarity(string1, string2)
print(similarity_score)  # Output: A value between 0 and 1 indicating similarity.


In [ ]:
def jaccard_similarity(str1, str2):
    set1, set2 = set(str1.split()), set(str2.split())
    return len(set1 & set2) / len(set1 | set2)

## `compute_jaccard_similarities(processed_value, compare_array)`

Compute Jaccard similarities between a processed value and an array of comparison texts.

### Parameters:

- `processed_value` (string): The processed value for which Jaccard similarities will be computed against the comparison texts.
- `compare_array` (array-like): An array of comparison texts.

### Description:

This function calculates Jaccard similarities between a single processed value and an array of comparison texts. It utilizes the previously defined `jaccard_similarity` function to measure the similarity between the processed value and each text in the comparison array.

### Parameters:

- `processed_value` (string): The processed value for which Jaccard similarities will be computed against the comparison texts.
- `compare_array` (array-like): An array of comparison texts.

### Output:

The function returns a NumPy array containing Jaccard similarity scores between the `processed_value` and each text in the `compare_array`. Each score is a value between 0 and 1, indicating the similarity between the processed value and a specific comparison text.

### Example Usage:

```python
# Call the compute_jaccard_similarities function
processed_value = "apple banana orange"
comparison_texts = ["banana orange kiwi", "grapes orange apple", "kiwi lemon lime"]
similarities = compute_jaccard_similarities(processed_value, comparison_texts)
print(similarities)  # Output: NumPy array of similarity scores.


In [ ]:
def compute_jaccard_similarities(processed_value, compare_array):
    return np.array([jaccard_similarity(processed_value, compare_text) for compare_text in compare_array])

## `process_row(row, columns_names, compare_tfidf, vectorizer, threshold, compare_array)`

Process a row of data and calculate hybrid similarities for multiple columns.

### Parameters:

- `row` (pandas Series): The row of data to be processed.
- `columns_names` (list): List of column names to process.
- `compare_tfidf` (scipy.sparse matrix): TF-IDF matrix for comparison texts.
- `vectorizer` (TfidfVectorizer): TF-IDF vectorizer instance used for transforming processed values.
- `threshold` (float): Similarity threshold for considering matches.
- `compare_array` (array-like): An array of comparison texts.

### Description:

This function processes a row of data by calculating hybrid similarities between the values in the specified columns and the comparison texts. It uses a combination of cosine similarity and Jaccard similarity to determine the level of similarity. The most similar value from the comparison array is identified and returned if the similarity score exceeds the threshold.

### Parameters:

- `row` (pandas Series): The row of data to be processed.
- `columns_names` (list): List of column names to process.
- `compare_tfidf` (scipy.sparse matrix): TF-IDF matrix for comparison texts.
- `vectorizer` (TfidfVectorizer): TF-IDF vectorizer instance used for transforming processed values.
- `threshold` (float): Similarity threshold for considering matches.
- `compare_array` (array-like): An array of comparison texts.

### Output:

The function returns a list of tuples, each containing the following information:

- Column name: The name of the processed column.
- Original value: The value from the processed row.
- Most similar value: The most similar value from the comparison array.
- Similarity score: The hybrid similarity score, expressed as a percentage.

### Example Usage:

```python
# Call the process_row function
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample data setup (replace with actual data)
data = pd.DataFrame({'column1': ['apple banana', 'orange kiwi'],
                     'column2': ['grapes pear', 'kiwi lemon']})

compare_texts = ["banana orange kiwi", "grapes orange apple", "kiwi lemon lime"]
compare_tfidf = vectorizer.transform(compare_texts)
threshold = 0.5

results = process_row(data.iloc[0], ['column1', 'column2'], compare_tfidf, vectorizer, threshold, compare_texts)
print(results)  # Output: List of similarity results for each column.


In [ ]:
def process_row(row, columns_names, compare_tfidf, vectorizer, threshold, compare_array):
    results = []
    for column_name in columns_names:
        value = row[column_name]
        processed_value = preprocess(value)
        if processed_value == '':
            continue

        tfidf_value = vectorizer.transform([processed_value])
        cosine_sim = cosine_similarity(tfidf_value, compare_tfidf, dense_output=True)[0]
        jaccard_sim = compute_jaccard_similarities(processed_value, compare_array)

        if cosine_sim.shape != jaccard_sim.shape:
            continue

        hybrid_similarity = (cosine_sim + jaccard_sim) / 2
        max_similarity_index = np.argmax(hybrid_similarity)
        similarity_score = hybrid_similarity[max_similarity_index]

        if similarity_score < threshold:
            continue

        most_similar_value = compare_array[max_similarity_index]
        results.append((column_name, value, most_similar_value, f'{similarity_score * 100:.2f}%'))
    return results


## `process_chunk(chunk, columns_names, compare_tfidf, vectorizer, threshold, compare_array)`

Process a chunk of data and calculate hybrid similarities for multiple rows.

### Parameters:

- `chunk` (pandas DataFrame): A chunk of data to be processed.
- `columns_names` (list): List of column names to process.
- `compare_tfidf` (scipy.sparse matrix): TF-IDF matrix for comparison texts.
- `vectorizer` (TfidfVectorizer): TF-IDF vectorizer instance used for transforming processed values.
- `threshold` (float): Similarity threshold for considering matches.
- `compare_array` (array-like): An array of comparison texts.

### Description:

This function processes a chunk of data by iterating through the rows and calculating hybrid similarities between the values in the specified columns and the comparison texts. It utilizes the previously defined `process_row` function to compute similarities for each row within the chunk.

### Parameters:

- `chunk` (pandas DataFrame): A chunk of data to be processed.
- `columns_names` (list): List of column names to process.
- `compare_tfidf` (scipy.sparse matrix): TF-IDF matrix for comparison texts.
- `vectorizer` (TfidfVectorizer): TF-IDF vectorizer instance used for transforming processed values.
- `threshold` (float): Similarity threshold for considering matches.
- `compare_array` (array-like): An array of comparison texts.

### Output:

The function returns a list of tuples, each containing the similarity results for each row within the chunk. Each tuple includes the following information:

- Column name: The name of the processed column.
- Original value: The value from the processed row.
- Most similar value: The most similar value from the comparison array.
- Similarity score: The hybrid similarity score, expressed as a percentage.

### Example Usage:

```python
# Call the process_chunk function
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Sample data setup (replace with actual data)
data = pd.DataFrame({'column1': ['apple banana', 'orange kiwi'],
                     'column2': ['grapes pear', 'kiwi lemon']})

compare_texts = ["banana orange kiwi", "grapes orange apple", "kiwi lemon lime"]
compare_tfidf = vectorizer.transform(compare_texts)
threshold = 0.5

chunk_size = 1000
chunk_generator = pd.read_csv('large_data.csv', chunksize=chunk_size)

for chunk in chunk_generator:
    results = process_chunk(chunk, ['column1', 'column2'], compare_tfidf, vectorizer, threshold, compare_texts)
    # Process or save the results as needed


In [ ]:
def process_chunk(chunk, columns_names, compare_tfidf, vectorizer, threshold, compare_array):
    print(f"Starting processing of chunk...")
    results = []
    for index, row in tqdm(chunk.iterrows(), total=chunk.shape[0]):
        results.extend(process_row(row, columns_names, compare_tfidf, vectorizer, threshold, compare_array))

    print(f"Finished processing of chunk.")
    return results


## `fit_vectorizer(compare_array)`

Fit a TF-IDF vectorizer on a given array of texts.

### Parameters:

- `compare_array` (array-like): An array of texts to fit the TF-IDF vectorizer.

### Description:

This function fits a TF-IDF vectorizer on a provided array of texts. The vectorizer learns the vocabulary and calculates the TF-IDF weights for the words in the texts, enabling further text processing and similarity calculations.

### Parameters:

- `compare_array` (array-like): An array of texts to fit the TF-IDF vectorizer.

### Output:

The function returns a tuple containing the following:

- `vectorizer` (TfidfVectorizer): The fitted TF-IDF vectorizer instance.
- `compare_tfidf` (scipy.sparse matrix): The TF-IDF matrix for the comparison texts after transformation.

### Example Usage:

```python
# Call the fit_vectorizer function
compare_texts = ["banana orange kiwi", "grapes orange apple", "kiwi lemon lime"]
vectorizer, compare_tfidf = fit_vectorizer(compare_texts)
print(vectorizer)  # Output: Fitted TF-IDF vectorizer instance.
print(compare_tfidf)  # Output: TF-IDF matrix for comparison texts.


In [ ]:
def fit_vectorizer(compare_array):
    print("Fitting the TF-IDF vectorizer...")
    vectorizer = TfidfVectorizer()
    compare_tfidf = vectorizer.fit_transform(compare_array)
    return vectorizer, compare_tfidf


## `compare_values(compare_array, df, columns_names, chunk_size=500, threshold=0.1, parallel=True)`

Compare values in a DataFrame with a given array of texts using parallel processing.

### Parameters:

- `compare_array` (array-like): An array of comparison texts.
- `df` (pandas DataFrame): The DataFrame containing the values to compare.
- `columns_names` (list): List of column names to compare.
- `chunk_size` (int, optional): The number of rows to process per chunk. Default is 500.
- `threshold` (float, optional): Similarity threshold for considering matches. Default is 0.1.
- `parallel` (bool, optional): Whether to use parallel processing. Default is True.

### Description:

This function compares values in a DataFrame with a given array of comparison texts. It uses the provided `process_chunk` function to process data chunks, enabling efficient parallel or sequential comparison of values.

### Parameters:

- `compare_array` (array-like): An array of comparison texts.
- `df` (pandas DataFrame): The DataFrame containing the values to compare.
- `columns_names` (list): List of column names to compare.
- `chunk_size` (int, optional): The number of rows to process per chunk. Default is 500.
- `threshold` (float, optional): Similarity threshold for considering matches. Default is 0.1.
- `parallel` (bool, optional): Whether to use parallel processing. Default is True.

### Output:

The function returns a pandas DataFrame containing the following columns:

- `column_name`: The name of the processed column.
- `value`: The value from the processed row.
- `matched_value`: The most similar value from the comparison array.
- `similarity_percentage`: The hybrid similarity score, expressed as a percentage.

### Example Usage:

```python
# Call the compare_values function
import pandas as pd

# Sample data setup (replace with actual data)
data = pd.DataFrame({'column1': ['apple banana', 'orange kiwi'],
                     'column2': ['grapes pear', 'kiwi lemon']})

compare_texts = ["banana orange kiwi", "grapes orange apple", "kiwi lemon lime"]
result_df = compare_values(compare_texts, data, ['column1', 'column2'])
print(result_df)  # Output: DataFrame containing similarity results.


In [ ]:
def compare_values(compare_array, df, columns_names, chunk_size=500, threshold=0.1, parallel=True):
    print("Starting comparison of values...")
    compare_array = [preprocess(val) for val in compare_array]
    columns_names = [preprocess(col) for col in columns_names]  # Preprocess column_names as well
    vectorizer, compare_tfidf = fit_vectorizer(compare_array)

    total_rows = df.shape[0]
    chunks = np.array_split(df, np.ceil(total_rows / chunk_size))
    print(f"Total rows: {total_rows}, total chunks to process: {len(chunks)}.")
    results = []

    if parallel:
        print("Processing in parallel...")
        with ProcessPoolExecutor() as executor:
            futures = [executor.submit(process_chunk, chunk, columns_names, compare_tfidf, vectorizer, threshold, compare_array)
                       for chunk in tqdm(chunks)]
            for future in tqdm(futures, desc="Processing chunks"):
                results.extend(future.result())
    else:
        print("Processing sequentially...")
        for chunk in tqdm(chunks, desc="Processing chunks"):
            results.extend(process_chunk(chunk, columns_names, compare_tfidf, vectorizer, threshold, compare_array))

    print("Finished comparison of values.")
    result_df = pd.DataFrame(results, columns=['column_name', 'value', 'matched_value', 'similarity_percentage'])
    return result_df

In [ ]:
total_arrays = [Type_TP]+[INTERVENANT_OPTIONS]+[Typologie]+[Causes_Echecs]+[ADMIN_NAMES]+OPTIONS_MAP_Stringheader

In [ ]:
np_array = np.array(total_arrays)

In [ ]:
np_array_f = np.hstack(np_array)

In [ ]:
columns_names = ["Priorité","Périmètre","Typologie","Type TP","CDP SFR","CDP KEOS","HO/HNO","Heure TP","ADMIN", "Nom de l'admin","STIT","Intervenant Terrain","Statut TP","Responsable echec","Causes Echecs","Livraison CR","Cloture AP Axis","Statut Attachement STIT","Statut PV SFR"]

In [ ]:
result_df = compare_values(INTERVENANT_OPTIONS, df, [columns_names[11]])
threshold = 0.1
summary_report(result_df, threshold)

In [ ]:
result_df.head()

In [ ]:
result_df.to_excel('result.xlsx', index=False)